In [ ]:
'''clear to run, check with nate if issues'''


#import pandas for data processing
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
def fname_ls_builder(fin_path,initial=True):
  #Find all files in data directory
  from os import listdir
  from os.path import isfile, join
  if initial:
    return [f for f in listdir(fin_path) if isfile(join(fin_path,f)) and (not ('_ds' in f or '_r' in f))]
  else:
    return [f for f in listdir(fin_path) if (isfile(join(fin_path,f)) and ('_ds' in f or '_r' in f))]

def df_builder(fin_path,fname_ls):
  #this function imports data from CSV files to one large dataframe

  #create list to hold dataframes
  df_ls=[]

  #read in each file
  for i in fname_ls:
    temp_df=pd.read_csv(fin_path+i,index_col=0)
    #print(i,'loaded\n\n')
    df_ls.append(temp_df)

  #create one large df
  df=pd.concat(df_ls)
  #print(df.shape)
  return df

def split_data(X):
  #separate y from X
  y=X['label']
  X.drop(['label'],axis=1,inplace=True)
  #split into train and dev sets
  #print(X,'\n\n',y)
  return X,y


#set filepaths for import/export
pkin_path=r'/content/drive/My Drive/ML Spectroscopy/Data/Preprocessed/Peaks Only/Labeled/'

pk_fname_ls=fname_ls_builder(pkin_path)

pk_df=df_builder(pkin_path,pk_fname_ls)

#drop relative intensities
pk_df.drop(columns=[i for i in pk_df.columns.values if 'val' in i],inplace=True)

pk_df,y=split_data(pk_df)


In [ ]:
#split into train and dev sets
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(pk_df, y, test_size = 0.20, random_state = 0)
#print(X_train,'\n\n',X_dev,'\n\n',y_train,'\n\n',y_dev)

#build a df of synthetic data for the training set
pk_fname_ls2=fname_ls_builder(pkin_path,initial=False)
pk_df2=df_builder(pkin_path,pk_fname_ls2)

#create columns to compare similar samples
temp_pk=pk_df2.copy()
X_temp=X_train.copy()
temp_pk.index.name='temp'
X_temp.index.name='temp'
#print('test X_train', X_train,'\n\n\n\n')
temp_pk.reset_index(inplace=True)
X_temp.reset_index(inplace=True)
temp_pk.rename(columns={temp_pk.index.name:'temp'},inplace=True)
#print(X_temp,'\n\n',temp_pk,'\n\nX_train\n\n',X_train)

#print(X_temp['temp'].str.split('-',expand=True)[[0,1]])
X_temp[['file','line']]=X_temp['temp'].str.split('-',expand=True)[[0,1]]
temp_pk[['file','line']]=temp_pk['temp'].str.split('-',expand=True)[[0,1]]
for i in X_temp.index.values:
#for i in range(10):
  #print(X_temp.loc[i,['temp']])
  temp_df=temp_pk.loc[temp_pk['line']==X_temp.loc[i,['line']].values[0]]
  #print('temp should be equal to string above\n',temp_df)
  temp_df=temp_df.loc[temp_df['file'].str.contains(X_temp.loc[i,['file']].values[0])]
  
  temp_df.set_index(keys=temp_df['temp'],drop=True,inplace=True)
  temp_df,temp_y=split_data(temp_df.set_index(keys=temp_df.temp,drop=True).drop(columns=['file','line','temp']))
  X_train.append(temp_df)
  #print(X_train)
  y_train.append(temp_y)
  if i%1000==0:
    print(i)

print(X_train,'\n\n',y_train)


In [ ]:
#scale data((X-mean)/std_dev)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_dev_sc = sc.fit_transform(X_dev)
#print(X.shape)

(225162, 5)


In [ ]:
from sklearn.decomposition import PCA
#run PCA
X_train_p=PCA(n_components=3).fit_transform(X_train_sc)
X_dev_p=PCA(n_components=3).fit_transform(X_dev_sc)
#print(X_transformed)

In [ ]:
#create model
from sklearn.linear_model import LogisticRegression 
model=LogisticRegression().fit(X_train_sc,y_train)
modelp=LogisticRegression().fit(X_train_p,y_train)

In [ ]:
#check model
print(model.score(X_dev_sc,y_dev))
print(modelp.score(X_dev_p,Py_dev))

0.8185552816823218
0.7579330713032665


In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y,model.predict(X_dev_sc))
pcm=confusion_matrix(y,modelp.predict(X_dev_p))
print(cm,'\n\n\n',pcm)

[[53500  3331    53  1921  1387     0]
 [ 4328 30571     5   158   188     0]
 [  204   217 16213  1115   459 12116]
 [  126   178  2104 30085  1992  1071]
 [ 2408  1526    32  1088 25186     0]
 [    2    50  4255   298   106 28889]] 


 [[52208  3149   433  1103  1121  2178]
 [ 4274 21588   100   127   181  8980]
 [  487   980 13545  2131   403 12778]
 [    4   658  1189 29188  2135  2382]
 [  623  2887  1075   279 25365    11]
 [ 1936   316  1945   396    79 28928]]


In [ ]:
testin_path=r'/content/drive/My Drive/ML Spectroscopy/Data/Preprocessed/Peaks Only/Unlabeled/'
test_ls=fname_ls_builder(testin_path)
test_df=df_builder(testin_path,test_ls)
test_df.drop(columns=[i for i in test_df.columns.values if 'val' in i],inplace=True)
X_test=sc.fit_transform(test_df)
X_test_p=PCA(n_components=3).fit_transform(X_test)

saved_data=r'/content/drive/My Drive/ML Spectroscopy/Programs/Data Processing/Saved Lists/'

pd.DataFrame(data=model.predict_proba(X_test),index=test_df.index.values,columns=sorted(y.unique())).to_csv(saved_data+r'log_reg.csv')
pd.DataFrame(data=modelp.predict_proba(X_test_p),index=test_df.index.values,columns=sorted(y.unique())).to_csv(saved_data+r'log_reg_pca.csv')
